# 影像圖片分類深度學習模型 - MLP (mnist)

## 模型與學習參數調整
進行下列調整後並比較結果
- model1 原本的模型
- model2 將隱藏層神經元個數增加到 784 個
- model3 新增一層隱藏層，神經元個數 256, 256 個
- model4 增加 Dropout 層 (50%)

## <font color='BLUE'>大綱:</font>
1. 載入影像資料集及預處理
2. 各種 MLP 模型參數
3. 訓練與分析


# 1. 載入影像資料集及預處理

In [ ]:
# 匯入模組
import tensorflow
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# plt.style.use('classic') #設定主題配色

from tensorflow.python.keras.utils import np_utils  # label 轉為 one-hot-encoding
from tensorflow.keras.models import Sequential  # 序列模型工具
from tensorflow.keras.layers import Dense  # 全連接層
from tensorflow.keras.layers import Dropout  # Drouout 層

np.random.seed(10)

# 載入手寫數字數據集
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 數據預處理 1.將二維矩陣轉成一維，image像素整數int轉成浮點數float
x_train_new = x_train.reshape(60000, 28*28).astype('float32')
x_test_new = x_test.reshape(10000, 28*28).astype('float32')

# 數據預處理 2.標準化0~1浮點數 Normalization
x_train_norm = x_train_new / 255
x_test_norm = x_test_new / 255

# One-Hot Encoding
y_train_onehot = np_utils.to_categorical(y_train)  #將 training 的 label 進行 one-hot encoding
y_test_onehot = np_utils.to_categorical(y_test) # 將測試的 labels 進行 one-hot encoding

print(x_train_norm.shape)
print(y_train_onehot.shape)
print(x_test_norm.shape)
print(y_test_onehot.shape)

# 2. 各種 MLP 模型參數

### 建立模型 多層感知模型MLP(Multilayer Perceptron)

輸入層 (x) 共有 28x28=784 個神經元

輸出層 (y) 共有 10 個 神經元(分10類) One-hot encoding


In [ ]:
# 建立模型(1)
model1 = Sequential(name='MLP-1')
model1.add(Dense(units=256, input_dim=784, kernel_initializer='normal', activation='relu'))
model1.add(Dense(units=10, kernel_initializer='normal', activation='softmax'))

# 建立模型(2)
model2 = Sequential(name='MLP-2')
model2.add(Dense(units=784, input_dim=784, kernel_initializer='normal', activation='relu'))
model2.add(Dense(units=10, kernel_initializer='normal', activation='softmax'))

# 建立模型(3)
model3 = Sequential(name='MLP-3')
model3.add(Dense(units=256, input_dim=784, kernel_initializer='normal', activation='relu'))
model3.add(Dense(units=256, kernel_initializer='normal', activation='relu'))
model3.add(Dense(units=10, kernel_initializer='normal', activation='softmax'))

# 建立模型(4)
model4 = Sequential(name='MLP-3')
model4.add(Dense(units=256, input_dim=784, kernel_initializer='normal', activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(units=256, kernel_initializer='normal', activation='relu'))
model4.add(Dense(units=10, kernel_initializer='normal', activation='softmax'))


In [ ]:
# model1.summary()
# model2.summary()
# model3.summary()
# model4.summary()

In [ ]:
# 編譯模型
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 3. 訓練與分析

### 請依序進行 model1 到 model4 的訓練，並記錄結果

In [ ]:
# 每一訓練週期 epoch 會計算 accuracy 並記錄在 history
model = model4
history = model.fit(x=x_train_norm, y=y_train_onehot, validation_split=0.2, epochs=10, batch_size=200, verbose=2)

### 建立函數 show_history 顯示訓練過程



In [ ]:
# 使用自訂函數讀取 history 以圖表顯示訓練過程
def show_history(history, train, validation):
    fig = plt.gcf()
    fig.set_size_inches(8,3)
    plt.plot(history.history[train])  #訓練過程
    plt.plot(history.history[validation])  #驗證過程
    plt.title('Training History')  #標題
    plt.ylabel(train)   #縱軸是accuracy 與 loss 記錄
    plt.xlabel('Epoch')  #橫軸是訓練週期
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

### 觀察分析訓練結果

In [ ]:
# 比較訓練 loss 與驗證 loss
show_history(history, 'loss', 'val_loss')
show_history(history, 'accuracy', 'val_accuracy')

### 測試及預測準確率


In [ ]:
# 評估模型 x_test_norm
scores = model.evaluate(x_test_norm, y_test_onehot)
print('測試集準確率 = {:2.2f} %'.format(scores[1]*100.0))

# 預測分類 x_test_norm
prediction_prob = model.predict(x_test_norm)
prediction = prediction_prob.argmax(axis = -1)

num_right = sum(prediction==y_test)
num_wrong = sum(prediction!=y_test)
ratio = (10000-num_wrong) / 10000
print('正確數:', num_right, '\t錯誤數:', num_wrong, '\t正確率:', ratio)